In [7]:
'''
參考https://github.com/Asoul/tsec
'''
## -*- coding: utf-8 -*-
#-*- coding: cp950 -*-
import os
import re
import csv
import logging
import requests
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
'''
定義Fuction
'''

#清除抓到的股票每個欄位中多餘的逗號還有空白
def clean_row(row):
    ''' Clean comma and spaces '''
    for index, content in enumerate(row):
        row[index] = re.sub(",", "", content.strip())
    return row

#紀錄stock_id這隻股票的資料以append的方式增加在stock_id.csv後面
def record(stock_id, row):
    ''' Save row to csv file '''
    prefix="data"
    f = open('../{}/{}.csv'.format(prefix, stock_id.strip()), 'a')
    cw = csv.writer(f, lineterminator='\n')
    cw.writerow(row)
    f.close()
    
#初始化csv檔的column
def Initialize(Stock_ID):
    #初始化Stock_ID.csv
    prefix = "../data"
    if not os.path.isfile('{}/{}.csv'.format(prefix, Stock_ID.strip())):
        ''' Save row to csv file '''
        f = open('{}/{}.csv'.format(prefix, Stock_ID.strip()), 'a', encoding='utf-8-sig')
        cw = csv.writer(f, lineterminator='\n')
        csv_Columns = ['日期','成交股數','成交金額','開盤價','最高價','最低價',
                       '收盤價','漲跌價差','成交筆數','最後揭示買價','最後揭示賣價',]
        cw.writerow(csv_Columns)
        f.close()
        
#爬Stock_ID這支股票指定日期Day這天的資料
def Get_TSEdata(Day, Stock_ID):
    #設定網頁中要輸入的選項，再設定要儲存的日期格式
    Date_str = '{0}{1:02d}{2:02d}'.format(Day.year, Day.month, Day.day)
    Store_time = '{0}-{1:02d}-{2:02d}'.format(Day.year, Day.month, Day.day)
    #print(Date_str)
    url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX'
    query_params = {
        'date': Date_str,
        'response': 'json',
        'type': 'ALL',
        '_': str(round(time.time() * 1000) - 500)
    }
    # Get json data
    page = requests.get(url, params=query_params)
    for ID in Stock_ID:
        Initialize(ID)
    if not page.ok:
        logging.error("Can not get TSE data at {}".format(Date_str))
    else:
        # Parse page
        content = page.json()
        #print(content)
        for attri in content:
            #print(attri)
            if attri == 'data5' or attri == 'data4' or attri == 'data3' or  attri == 'data2' or attri == 'data1':
                for data in content[attri]:
                    for ID in Stock_ID:
                        if str(data[0]).strip() == ID:
                            sign = '-' if data[9].find('green') > 0 else ''
                            #print(data)
                            row = [
                                Store_time, # 日期
                                data[2], # 成交股數
                                data[4], # 成交金額
                                data[5], # 開盤價
                                data[6], # 最高價
                                data[7], # 最低價
                                data[8], # 收盤價
                                sign + data[10], # 漲跌價差
                                data[3], # 成交筆數
                                data[11],#最後揭示買價
                                data[13],#最後揭示賣價
                            ]
                            #print(row)
                            clean_row(row)
                            print(row)
                            record(data[0], row)

'''
抓取Stock_ID在日期範圍(First_Day,Last_Day)內的所有資料
['日期','成交股數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數','最後揭示買價','最後揭示賣價']
'''
#預設抓台積電(2330)從今天到2004,2,11的資料
def Get_Stock_DATA(Stock_ID = ["2330"], First_Day = datetime.today(), Last_Day = datetime(2004,2,11)):
    #Set Stock_ID that need to crawl
    print("Crawl " + str(Stock_ID) + "Stock Data")
    #Set logging
    if not os.path.isdir('../log'):
        os.makedirs('../log')
    logging.basicConfig(filename='../log/crawl-error.log',
                        level=logging.ERROR,
                        format='%(asctime)s\t[%(levelname)s]\t%(message)s',
                        datefmt='%Y/%m/%d %H:%M:%S')
    #Make directory if not exist when initialize
    prefix='../data'
    if not os.path.isdir(prefix):
        os.mkdir(prefix)
        
    #The First_Day and Last_Day
    Date_str = '{0}/{1:02d}/{2:02d}'.format(First_Day.year - 1911, First_Day.month, First_Day.day)
    CE_Date_str = '{0}/{1:02d}/{2:02d}'.format(First_Day.year, First_Day.month, First_Day.day)
    print("Start on " + Date_str)
    print("Start on " + CE_Date_str)
    Date_str = '{0}/{1:02d}/{2:02d}'.format(Last_Day.year - 1911, Last_Day.month, Last_Day.day)
    CE_Date_str = '{0}/{1:02d}/{2:02d}'.format(Last_Day.year, Last_Day.month, Last_Day.day)
    print("End of " + Date_str)
    print("End of " + CE_Date_str)
    
    #Set Max_Error to 5
    Max_Error = 5
    Error_Times = 0
    #Crawl stock until Last_Day
    while Error_Times < Max_Error and First_Day >= Last_Day:
        try:
            Get_TSEdata(Last_Day, Stock_ID)
            Error_Times = 0
        except:
            '''When crawl data occuring problem add one to Error_Times'''
            date_str = Last_Day.strftime('%Y/%m/%d')
            logging.error('Crawl raise error {}'.format(date_str))
            Error_Times += 1
            continue
        finally:
            Last_Day += timedelta(1)
            
    print("Finish")

#每日更新，去讀取資料夾內的StockID更新
def DailyUpdate():
    #找../data裡面有的股票資料去更新
    DataList = os.listdir("../data")
    print(DataList[1:])
    print("Update...")
    for CSV in DataList[1:]:
        #讀檔去找最後一筆的日期
        print("{}".format(CSV[:-4]))
        df = pd.read_csv("../data/{}.csv".format(CSV[:-4]))
        date = np.array(df.iloc[df.shape[0]-1:df.shape[0],:1])

        strdate = str(date)
        #最後一筆的日期
        year = int(strdate[3:7])
        month = int(strdate[8:10])
        day = int(strdate[11:13])
        Stock_ID = []
        Stock_ID.append(CSV[:-4])
        
        LastUpdate = datetime(year,month,day) + timedelta(1)
        Today = datetime.today()
        if LastUpdate != Today:
            LastUpdate_str = '{0}/{1:02d}/{2:02d}'.format(LastUpdate.year, LastUpdate.month, LastUpdate.day)
            Today_str = '{0}/{1:02d}/{2:02d}'.format(Today.year, Today.month, Today.day)
            print("From {} to {} (Today)".format(LastUpdate_str, Today_str))
            
            #最多Error 5次
            Max_Error = 5
            Error_Times = 0
            #Crawl stock until Last_Day
            while Error_Times < Max_Error and Today >= LastUpdate:
                try:
                    Get_TSEdata(LastUpdate, Stock_ID)
                    Error_Times = 0
                except:
                    #When crawl data occuring problem add one to Error_Times
                    date_str = LastUpdate.strftime('%Y/%m/%d')
                    logging.error('Crawl raise error {}'.format(date_str))
                    Error_Times += 1
                    continue
                finally:
                    LastUpdate += timedelta(1)
        else:
            print("Nothing to Update")

#回傳資料夾data內沒有的Stock_ID，可以藉由這個抓取沒在資料庫內的股票
def CheckCSV(Stock_ID):
    prefix = "../data"
    CrawlID = []
    for ID in Stock_ID:
        if not os.path.isfile('{}/{}.csv'.format(prefix, ID.strip())):
            CrawlID.append(ID)
    #Get_Stock_DATA(CrawlID)
    return CrawlID


#Stock_ID = ["2330","2002","3008",'2332']
#Stock_ID = ["2332"]
#datetime(2004,2,11) data2
#datetime(2009,1,5) data4
#datetime(2011,8,1) data5
#Get_Stock_DATA(Stock_ID, Last_Day = datetime.today() - timedelta(10))
#Get_Stock_DATA(Stock_ID=Stock_ID)
#Get_TSEdata(datetime(2009,1,5), ['3008'])

In [8]:
DailyUpdate()
Stock_ID = ['2330','2002','3008','2332','1560','2317','3481','2409']
CrawlID = CheckCSV(Stock_ID)
CrawlID

['2002.csv', '2330.csv', '2332.csv', '3008.csv']
Update...
2002
From 2017/07/08 to 2017/07/15 (Today)
['2017-07-10', '12479305', '316815300', '25.50', '25.55', '25.20', '25.30', '-0.25', '5119', '25.30', '25.35']
['2017-07-11', '12625362', '321510212', '25.35', '25.60', '25.35', '25.55', '0.25', '4446', '25.50', '25.55']
['2017-07-12', '16226281', '415386584', '25.55', '25.80', '25.50', '25.55', '0.00', '5860', '25.50', '25.55']
['2017-07-13', '20154791', '516622398', '25.60', '25.70', '25.50', '25.55', '0.00', '5182', '25.55', '25.60']
['2017-07-14', '12210971', '313006237', '25.65', '25.70', '25.60', '25.60', '0.05', '4398', '25.55', '25.60']
2330
From 2017/07/15 to 2017/07/15 (Today)
2332
From 2017/07/15 to 2017/07/15 (Today)
3008
From 2017/07/15 to 2017/07/15 (Today)


['1560', '2317', '3481', '2409']

In [9]:
Get_Stock_DATA(Stock_ID=CrawlID)

Crawl ['1560', '2317', '3481', '2409']Stock Data
Start on 106/07/15
Start on 2017/07/15
End of 93/02/11
End of 2004/02/11
['2004-02-11', '7171842', '1059443894', '150.00', '150.00', '147.00', '148.00', '-2.00', '2707', '147.50', '148.00']
['2004-02-11', '117539208', '5397204749', '46.30', '46.80', '45.40', '45.50', '-2.60', '25572', '45.50', '45.60']
['2004-02-12', '6395493', '963318439', '148.50', '152.00', '148.50', '149.00', '1.00', '2583', '149.00', '149.50']
['2004-02-12', '67321596', '3077018505', '45.50', '46.20', '44.70', '45.60', '0.10', '14418', '45.50', '45.60']
['2004-02-13', '7237308', '1099536396', '152.00', '154.00', '149.50', '153.00', '4.00', '2990', '152.00', '153.00']
['2004-02-13', '42345678', '1946245276', '45.80', '46.30', '45.50', '45.90', '0.30', '8568', '45.90', '46.00']
['2004-02-16', '8059369', '1199633692', '150.00', '152.00', '148.00', '148.00', '-5.00', '3836', '148.00', '148.50']
['2004-02-16', '38685679', '1764296961', '45.90', '46.20', '45.20', '45.20',

['2004-04-01', '6450673', '931002862', '143.00', '146.00', '142.00', '146.00', '3.50', '2567', '145.50', '146.00']
['2004-04-01', '107843184', '6929012450', '63.00', '66.00', '62.50', '65.50', '2.00', '22753', '65.00', '65.50']
['2004-04-02', '6754562', '965827903', '146.00', '146.00', '142.00', '142.50', '-3.50', '2500', '142.50', '143.00']
['2004-04-02', '70950909', '4615187030', '66.00', '66.00', '64.50', '64.50', '-1.00', '13656', '64.50', '65.00']
['2004-04-05', '6193650', '892971519', '144.00', '145.50', '142.50', '145.00', '2.50', '2504', '144.50', '145.00']
['2004-04-05', '117928433', '7720008565', '65.00', '66.50', '64.00', '65.50', '1.00', '20001', '65.00', '65.50']
['2004-04-06', '10199971', '1496862734', '146.00', '148.50', '145.50', '145.50', '0.50', '3349', '145.50', '146.00']
['2004-04-06', '122516494', '7821007306', '66.00', '66.00', '62.50', '62.50', '-3.00', '27437', '62.50', '63.00']
['2004-04-07', '5254361', '769450868', '145.50', '147.50', '145.00', '147.50', '2.00

['2004-05-21', '92030980', '6036801366', '63.50', '67.00', '63.00', '66.50', '3.50', '20972', '66.00', '66.50']
['2004-05-24', '3491700', '461755087', '131.50', '133.50', '131.00', '133.00', '0.00', '1114', '132.00', '133.00']
['2004-05-24', '69232819', '4584631519', '66.00', '68.00', '65.00', '66.00', '-0.50', '14300', '66.00', '66.50']
['2004-05-25', '3076942', '409905439', '131.50', '134.00', '131.50', '134.00', '1.00', '1086', '133.50', '134.00']
['2004-05-25', '90143000', '6105354000', '66.00', '69.00', '66.00', '67.50', '1.50', '16787', '67.50', '68.00']
['2004-05-26', '4346161', '593633090', '136.00', '137.50', '136.00', '136.00', '2.00', '1751', '135.50', '136.00']
['2004-05-26', '155201000', '10721963500', '70.00', '70.50', '68.00', '68.50', '1.00', '27859', '68.50', '69.00']
['2004-05-27', '3036002', '405584412', '136.00', '137.00', '132.00', '133.00', '-3.00', '1260', '133.00', '133.50']
['2004-05-27', '148943000', '9672609500', '65.50', '66.00', '64.00', '64.50', '0.00', '3

['2004-07-13', '81360735', '3834061603', '47.00', '47.60', '46.50', '47.00', '-2.00', '19088', '46.90', '47.00']
['2004-07-14', '5655344', '678623124', '120.00', '121.00', '119.00', '121.00', '2.00', '1713', '121.00', '121.50']
['2004-07-14', '67411931', '3154043600', '47.00', '47.50', '46.20', '46.20', '-0.80', '17415', '46.20', '46.30']
['2004-07-15', '9289686', '1098225361', '119.00', '120.00', '116.50', '117.00', '-4.00', '2172', '116.50', '117.00']
['2004-07-15', '88215895', '3995892683', '45.80', '45.90', '44.90', '45.20', '-1.00', '20817', '45.20', '45.30']
['2004-07-16', '6325302', '737079697', '115.50', '118.00', '115.50', '116.00', '-1.00', '1640', '116.00', '116.50']
['2004-07-16', '66475745', '2993963949', '44.90', '45.70', '44.00', '44.00', '-1.20', '15979', '44.00', '44.10']
['2004-07-19', '5449433', '622111986', '116.00', '117.00', '111.50', '114.50', '-1.50', '1723', '114.00', '114.50']
['2004-07-19', '72252562', '3175270416', '43.80', '44.40', '43.10', '44.10', '0.10',

['2004-09-06', '7551082', '843151432', '113.00', '113.50', '110.00', '112.00', '-1.00', '2619', '112.00', '112.50']
['2004-09-06', '70787507', '3071795645', '43.90', '44.20', '42.70', '43.40', '-0.80', '16265', '43.40', '43.50']
['2004-09-07', '6248606', '714116163', '113.00', '115.00', '112.50', '115.00', '3.00', '2264', '114.50', '115.00']
['2004-09-07', '54002480', '2357118652', '43.40', '44.10', '43.20', '43.60', '0.20', '11806', '43.50', '43.60']
['2004-09-08', '8063072', '933515842', '115.00', '116.50', '114.50', '116.00', '1.00', '2666', '115.50', '116.00']
['2004-09-08', '87776627', '3851400635', '43.60', '44.50', '43.40', '43.40', '-0.20', '17330', '43.40', '43.50']
['2004-09-09', '3837473', '442058615', '116.00', '116.00', '114.50', '114.50', '-1.50', '1508', '114.50', '115.00']
['2004-09-09', '90019091', '3923423448', '43.00', '44.30', '42.90', '43.90', '0.50', '17175', '43.80', '43.90']
['2004-09-10', '9026233', '1056473552', '116.00', '118.00', '115.50', '117.00', '2.50', 

...

['2017-03-14', '888912', '65096929', '72.00', '74.00', '72.00', '73.50', '0.40', '583', '73.40', '73.60']
['2017-03-14', '20693907', '1866210292', '90.00', '90.50', '89.90', '90.00', '0.00', '9241', '90.00', '90.10']
['2017-03-14', '47083640', '564185941', '11.95', '12.05', '11.90', '11.95', '0.10', '6504', '11.95', '12.00']
['2017-03-14', '54820181', '666733935', '12.00', '12.30', '11.95', '12.15', '0.20', '7920', '12.15', '12.20']
['2017-03-15', '869375', '63459198', '74.00', '74.00', '72.20', '73.00', '-0.50', '667', '72.60', '73.00']
['2017-03-15', '17686039', '1594078973', '89.90', '90.40', '89.70', '90.10', '0.10', '7119', '90.00', '90.10']
['2017-03-15', '40267138', '483627229', '12.05', '12.10', '11.90', '11.95', '0.00', '7644', '11.90', '11.95']
['2017-03-15', '30625194', '374711121', '12.15', '12.35', '12.15', '12.15', '0.00', '5249', '12.15', '12.20']
['2017-03-16', '1127690', '83353466', '73.20', '74.60', '72.70', '74.30', '1.30', '802', '74.10', '74.30']
['2017-03-16', '38

['2017-04-12', '1657222', '115252925', '71.60', '71.70', '67.60', '69.30', '-2.50', '1100', '69.30', '69.40']
['2017-04-12', '40250876', '3907517072', '97.40', '97.40', '96.60', '97.30', '0.40', '12909', '97.20', '97.30']
['2017-04-12', '158439604', '2006517745', '12.50', '12.90', '12.40', '12.80', '0.45', '23560', '12.75', '12.80']
['2017-04-12', '295239391', '4121800849', '13.60', '14.45', '13.50', '14.25', '0.85', '35358', '14.20', '14.25']
['2017-04-13', '1858889', '126671460', '68.80', '69.20', '67.50', '67.80', '-1.50', '1258', '67.70', '67.90']
['2017-04-13', '61732380', '6081861240', '97.70', '99.30', '97.50', '98.50', '1.20', '24052', '98.40', '98.50']
['2017-04-13', '196604389', '2555816557', '12.90', '13.25', '12.75', '12.85', '0.05', '24213', '12.85', '12.90']
['2017-04-13', '292546582', '4216290661', '14.30', '14.80', '14.10', '14.20', '-0.05', '38741', '14.15', '14.20']
['2017-04-14', '1554085', '103178810', '67.50', '67.60', '65.50', '65.50', '-2.30', '999', '65.40', '65

['2017-05-10', '753776', '52929351', '70.10', '71.40', '69.40', '71.10', '1.90', '534', '71.10', '71.20']
['2017-05-10', '44982960', '4687034453', '105.50', '105.50', '103.00', '104.00', '0.00', '22963', '103.50', '104.00']
['2017-05-10', '85316374', '1001117702', '11.60', '11.85', '11.60', '11.75', '0.15', '11310', '11.70', '11.75']
['2017-05-10', '117551688', '1565736712', '13.15', '13.45', '13.15', '13.35', '0.25', '14000', '13.35', '13.40']
['2017-05-11', '547700', '38731420', '71.80', '71.80', '70.00', '70.50', '-0.60', '398', '70.50', '70.60']
['2017-05-11', '39791573', '4119466505', '104.00', '104.00', '103.00', '104.00', '0.00', '17492', '103.50', '104.00']
['2017-05-11', '118334194', '1369883709', '11.85', '11.85', '11.45', '11.45', '-0.30', '15519', '11.45', '11.50']
['2017-05-11', '342243874', '4558577212', '13.65', '13.75', '12.95', '12.95', '-0.40', '42171', '12.95', '13.00']
['2017-05-12', '647401', '46338829', '70.70', '72.00', '70.70', '71.50', '1.00', '475', '71.40', '

['2017-06-07', '626266', '45511470', '73.30', '73.70', '72.30', '72.70', '-0.50', '464', '72.70', '72.80']
['2017-06-07', '27310162', '2864892921', '105.00', '105.50', '104.50', '105.00', '0.00', '8026', '104.50', '105.00']
['2017-06-07', '177534642', '2226753077', '12.35', '12.65', '12.35', '12.55', '0.30', '19156', '12.55', '12.60']
['2017-06-07', '115702329', '1673258636', '14.45', '14.65', '14.30', '14.40', '0.05', '19448', '14.40', '14.45']
['2017-06-08', '742122', '54224592', '73.00', '73.70', '72.60', '72.80', '0.10', '510', '72.80', '73.00']
['2017-06-08', '15154346', '1592538830', '105.00', '105.50', '104.50', '105.00', '0.00', '6308', '105.00', '105.50']
['2017-06-08', '88498289', '1103517396', '12.65', '12.65', '12.40', '12.40', '-0.15', '11592', '12.40', '12.45']
['2017-06-08', '94897191', '1352614135', '14.45', '14.50', '14.15', '14.15', '-0.25', '15855', '14.15', '14.20']
['2017-06-09', '773665', '56428912', '73.40', '73.40', '72.60', '72.80', '0.00', '539', '72.80', '72.

['2017-07-04', '434300', '31857460', '73.00', '73.60', '73.00', '73.20', '0.50', '328', '73.20', '73.30']
['2017-07-04', '65450472', '7491628803', '116.50', '117.00', '113.00', '113.00', '-3.50', '29114', '113.00', '113.50']
['2017-07-04', '128561757', '1713819417', '13.55', '13.65', '13.10', '13.15', '-0.40', '18470', '13.15', '13.20']
['2017-07-04', '123791863', '1984708998', '16.25', '16.30', '15.85', '15.85', '-0.45', '16789', '15.85', '15.90']
['2017-07-05', '453630', '33439129', '73.60', '74.30', '73.20', '73.70', '0.50', '371', '73.60', '73.80']
['2017-07-05', '58318681', '6682942496', '113.50', '116.00', '113.00', '116.00', '3.00', '22150', '115.50', '116.00']
['2017-07-05', '75016560', '989896954', '13.25', '13.30', '13.05', '13.25', '0.10', '9782', '13.20', '13.25']
['2017-07-05', '78948850', '1260860300', '15.95', '16.10', '15.75', '16.10', '0.25', '11458', '16.05', '16.10']
['2017-07-06', '989146', '73546179', '73.80', '75.40', '72.80', '75.20', '1.50', '651', '75.10', '75.